In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [3]:
test = pd.read_csv('test.csv')
test

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\scart\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords

sw = stopwords.words('english')

In [7]:
sw.extend(["'d", "'ll", "'m", "'re","'ve", "'t", "'s", "n't"])

In [8]:
from nltk import WordNetLemmatizer

wnl = WordNetLemmatizer()

In [9]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\scart\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [10]:
from string import punctuation

In [11]:
def get_wordnet_pos(word):
    '''Получаем для слова POS-тэг, чтобы смочь его лемматизировать'''
    tag = nltk.pos_tag([word])[0][1].lower()[0]
    tags = {'v', 'n', 'a', 'r'}
    if tag == 'j':
        tag = 'a'
    if tag in tags:
        return tag
    else:
        return 'n'

In [12]:
def tokenize(text, lemmatize=True):
    '''Токенизирует текст, по умолчанию еще лемматизирует, но можно не лемматизировать'''
    words = word_tokenize(text)
    new_words = []
    for word in words:
      new_w = ''
      for ch in word:
        if ch.isalpha() or ch == "'":
          new_w += ch.lower()
      if new_w != '' and new_w not in sw:
          new_words.append(new_w)
    words = new_words
    if lemmatize:
        words = [wnl.lemmatize(w, pos=get_wordnet_pos(w)) for w in words]
    return words

In [13]:
train['lemmatized'] = None

In [14]:
test['lemmatized'] = None

In [17]:
for ind in train.index:
    text = train.at[ind, 'comment_text']
    train.at[ind, 'lemmatized'] = tokenize(text)

In [18]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,lemmatized
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[explanation, edits, make, username, hardcore,..."
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[d'aww, match, background, colour, seemingly, ..."
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, really, try, edit, war, guy, consta..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[ca, make, real, suggestion, improvement, wond..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page]"


In [19]:
train.to_csv('lemm_train.csv')

In [20]:
for ind in test.index:
    text = test.at[ind, 'comment_text']
    test.at[ind, 'lemmatized'] = tokenize(text)

In [22]:
test.to_csv('lemm_test.csv')

In [23]:
all = pd.concat([train, test], ignore_index=True)
all

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,lemmatized
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0.0,0.0,0.0,0.0,0.0,0.0,"[explanation, edits, make, username, hardcore,..."
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0.0,0.0,0.0,0.0,0.0,0.0,"[d'aww, match, background, colour, seemingly, ..."
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0.0,0.0,0.0,0.0,0.0,0.0,"[hey, man, really, try, edit, war, guy, consta..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0.0,0.0,0.0,0.0,0.0,0.0,"[ca, make, real, suggestion, improvement, wond..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0.0,0.0,0.0,0.0,0.0,0.0,"[sir, hero, chance, remember, page]"
...,...,...,...,...,...,...,...,...,...
312730,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu...",NaN,NaN,NaN,NaN,NaN,NaN,"[totally, agree, stuff, nothing, toolongcrap]"
312731,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...,NaN,NaN,NaN,NaN,NaN,NaN,"[throw, field, home, plate, get, faster, throw..."
312732,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ...",NaN,NaN,NaN,NaN,NaN,NaN,"[okinotorishima, category, see, change, agree,..."
312733,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the...",NaN,NaN,NaN,NaN,NaN,NaN,"['', one, found, nation, eu, germany, law, ret..."


In [2]:
train = pd.read_csv('lemm_train.csv')
train

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,lemmatized
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"['explanation', 'edits', 'make', 'username', '..."
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[""d'aww"", 'match', 'background', 'colour', 'se..."
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"['hey', 'man', 'really', 'try', 'edit', 'war',..."
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"['ca', 'make', 'real', 'suggestion', 'improvem..."
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"['sir', 'hero', 'chance', 'remember', 'page']"
...,...,...,...,...,...,...,...,...,...,...
159566,159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,"['second', 'time', 'ask', 'view', 'completely'..."
159567,159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,"['ashamed', 'horrible', 'thing', 'put', 'talk'..."
159568,159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,"['spitzer', 'umm', 'there', 'actual', 'article..."
159569,159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0,"['look', 'like', 'actually', 'put', 'speedy', ..."


In [13]:
def norm_lemm(text):
    text = text[1:-1]
    text = text.split("', '")
    text[0] = text[0][1:]
    text[-1] = text[-1][:-1]
    return ' '.join(text)

In [15]:
for i in train.index:
    train.at[i, 'lemmatized'] = norm_lemm(train.at[i, 'lemmatized'])

In [17]:
train.to_csv('lemm_train_final.csv')

In [18]:
test = pd.read_csv('lemm_test.csv')
test

,Unnamed: 0,id,comment_text,lemmatized
0,0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,"['yo', 'bitch', 'ja', 'rule', 'succesful', 'ev..."
1,1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,"['rfc', 'title', 'fine', 'imo']"
2,2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...","['source', 'zawe', 'ashton', 'lapland']"
3,3,00017563c3f7919a,":If you have a look back at the source, the in...","['look', 'back', 'source', 'information', 'upd..."
4,4,00017695ad8997eb,I don't anonymously edit articles at all.,"['anonymously', 'edit', 'article']"
...,...,...,...,...
153159,153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu...","['totally', 'agree', 'stuff', 'nothing', 'tool..."
153160,153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...,"['throw', 'field', 'home', 'plate', 'get', 'fa..."
153161,153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ...","['okinotorishima', 'category', 'see', 'change'..."
153162,153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the...","[""''"", 'one', 'found', 'nation', 'eu', 'german..."


In [19]:
for i in test.index:
    test.at[i, 'lemmatized'] = norm_lemm(test.at[i, 'lemmatized'])

In [20]:
test

,Unnamed: 0,id,comment_text,lemmatized
0,0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,yo bitch ja rule succesful ever whats hat sad ...
1,1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,rfc title fine imo
2,2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",source zawe ashton lapland
3,3,00017563c3f7919a,":If you have a look back at the source, the in...",look back source information update correct fo...
4,4,00017695ad8997eb,I don't anonymously edit articles at all.,anonymously edit article
...,...,...,...,...
153159,153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu...",totally agree stuff nothing toolongcrap
153160,153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...,throw field home plate get faster throw cut ma...
153161,153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ...","okinotorishima category see change agree', ""''..."
153162,153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the...","''"", 'one found nation eu germany law return q..."


In [21]:
test.to_csv('lemm_test_final.csv')